In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/aerial-cactus-identification/sample_submission.csv
/kaggle/input/aerial-cactus-identification/train.zip
/kaggle/input/aerial-cactus-identification/test.zip
/kaggle/input/aerial-cactus-identification/train.csv


In [1]:
!unzip -q -o /kaggle/input/aerial-cactus-identification/train.zip
!unzip -q -o /kaggle/input/aerial-cactus-identification/test.zip

In [4]:
train=pd.read_csv('/kaggle/input/aerial-cactus-identification/train.csv')
train

,id,has_cactus
0,0004be2cfeaba1c0361d39e2b000257b.jpg,1
1,000c8a36845c0208e833c79c1bffedd1.jpg,1
2,000d1e9a533f62e55c289303b072733d.jpg,1
3,0011485b40695e9138e92d0b3fb55128.jpg,1
4,0014d7a11e90b62848904c1418fc8cf2.jpg,1
...,...,...
17495,ffede47a74e47a5930f81c0b6896479e.jpg,0
17496,ffef6382a50d23251d4bc05519c91037.jpg,1
17497,fff059ecc91b30be5745e8b81111dc7b.jpg,1
17498,fff43acb3b7a23edcc4ae937be2b7522.jpg,0


In [5]:
train['path']='train/'+train['id']
train

,id,has_cactus,path
0,0004be2cfeaba1c0361d39e2b000257b.jpg,1,train/0004be2cfeaba1c0361d39e2b000257b.jpg
1,000c8a36845c0208e833c79c1bffedd1.jpg,1,train/000c8a36845c0208e833c79c1bffedd1.jpg
2,000d1e9a533f62e55c289303b072733d.jpg,1,train/000d1e9a533f62e55c289303b072733d.jpg
3,0011485b40695e9138e92d0b3fb55128.jpg,1,train/0011485b40695e9138e92d0b3fb55128.jpg
4,0014d7a11e90b62848904c1418fc8cf2.jpg,1,train/0014d7a11e90b62848904c1418fc8cf2.jpg
...,...,...,...
17495,ffede47a74e47a5930f81c0b6896479e.jpg,0,train/ffede47a74e47a5930f81c0b6896479e.jpg
17496,ffef6382a50d23251d4bc05519c91037.jpg,1,train/ffef6382a50d23251d4bc05519c91037.jpg
17497,fff059ecc91b30be5745e8b81111dc7b.jpg,1,train/fff059ecc91b30be5745e8b81111dc7b.jpg
17498,fff43acb3b7a23edcc4ae937be2b7522.jpg,0,train/fff43acb3b7a23edcc4ae937be2b7522.jpg


In [6]:
train['has_cactus']=train['has_cactus'].astype('str')

In [8]:
#folder 만들어서 image 넣기 
from tqdm import tqdm
import cv2
os.makedirs('resized_train',exist_ok=True)
for path in tqdm(train['path']):
    image=cv2.imread(path)
    image=cv2.resize(image,(256,256))
    label=path.split('/')[-2]
    name=path.split('/')[-1]
    cv2.imwrite('resized_train/'+label+'_'+name,image)
    



100%|██████████| 17500/17500 [00:27<00:00, 628.89it/s]


In [9]:
import glob
train=pd.DataFrame({'path':glob.glob("resized_train/*")})
train

,path
0,resized_train/train_f80ce866aa8d947e04edb87e5f...
1,resized_train/train_d81e3689b0f6fd2a3cb25eae30...
2,resized_train/train_c65dbf591bd3757a135fc610ac...
3,resized_train/train_21e59f733351d6181bb3e8289b...
4,resized_train/train_c678322054fadd5b26f8069827...
...,...
17495,resized_train/train_e2607a8c29b009228f77c2ebe1...
17496,resized_train/train_b5cffadf5aee2e25742bf17ea3...
17497,resized_train/train_8303d7cac6a95ba3a34ecf6200...
17498,resized_train/train_0052d90950c3f08ed778d638a9...


In [10]:
train['has_cactus']=train['path'].apply(lambda x:x.split('/')[-1].split('_')[0])
train


,path,has_cactus
0,resized_train/train_f80ce866aa8d947e04edb87e5f...,train
1,resized_train/train_d81e3689b0f6fd2a3cb25eae30...,train
2,resized_train/train_c65dbf591bd3757a135fc610ac...,train
3,resized_train/train_21e59f733351d6181bb3e8289b...,train
4,resized_train/train_c678322054fadd5b26f8069827...,train
...,...,...
17495,resized_train/train_e2607a8c29b009228f77c2ebe1...,train
17496,resized_train/train_b5cffadf5aee2e25742bf17ea3...,train
17497,resized_train/train_8303d7cac6a95ba3a34ecf6200...,train
17498,resized_train/train_0052d90950c3f08ed778d638a9...,train


In [ ]:
train['has_cactus']=train['has_cactus'].astype(str)

from sklearn.model_selection import train_test_split
#x_train 이랑 x_valid 만 넣는 이유는 train_generator 가 알아서 y 값을 넣어야줘야한다 
x_train,x_valid=train_test_split(train,test_size=0.2,random_state=42)

from tensorflow.keras.preprocessing.image import ImageDataGenerator
idg=ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

#idg 에 train 에 만 넣어도
idg2=ImageDataGenerator()

train_generator=idg.flow_from_dataframe(x_train,x_col='path',y_col='has_cactus')
valid_generator=idg2.flow_from_dataframe(x_valid,x_col='path',y_col='has_cactus')





In [11]:
from tensorflow.keras import *
from tensorflow.keras.layers import *
from tensorflow.keras.applications import EfficientNetB1
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

2024-03-12 02:06:43.885951: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-12 02:06:43.886071: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-12 02:06:44.177750: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [12]:
model = Sequential()
model.add(EfficientNetB1(include_top=False, pooling='avg'))
model.add(Dense(2, activation='softmax'))  
model.compile(metrics=['acc'], optimizer='adam', loss='categorical_crossentropy')

27018416/27018416 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [13]:
# Add the EarlyStopping callback
early_stopping = EarlyStopping(
    patience=2,  # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True  # Whether to restore model weights from the epoch with the best value of the monitored quantity
)

# Fit the model using both the training and validation generators
model.fit(
    train_generator,
    epochs=20,
    validation_data=valid_generator,
    callbacks=[early_stopping]  # Include the EarlyStopping callback here
)


NameError: name 'train_generator' is not defined

In [ ]:
#평가를 할때 annotations데이터 섞이는걸 방지하기위해 항상 sample submission 사용
test=pd.read_csv('/kaggle/input/aerial-cactus-identification/sample_submission.csv')
test

In [ ]:
test['path']='test/'+test['id']

In [ ]:
test_generator=idg2.flow_from_dataframe(test, x_col='path', y_col=None, shuffle=False, class_mode=None )

In [ ]:
result=model.predict(test_generator)
result

In [ ]:
sub=pd.read_csv('/kaggle/input/aerial-cactus-identification/sample_submission.csv')
sub['has_cactus']=result[:,1]
sub

In [ ]:
sub.to_csv('sub123.csv',index=0)